In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv('articles.csv',  sep='|')
data.columns = ['url', 'headline', 'paragraphs']
data['paragraphs'] = data['paragraphs'].fillna('')

In [34]:
tfidf = TfidfVectorizer(ngram_range=(1,1))

In [35]:
tfidf_matrix = tfidf.fit_transform(data['paragraphs'])

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
indices = pd.Series(data.index, index=data['url']).drop_duplicates()

In [9]:
indexes = [1, 3, 5, 7, 8]

In [14]:
dfs = [pd.DataFrame(cosine_sim[index]) for index in indexes]
sim_scores = pd.concat(dfs, axis=1)    

In [18]:
average_scores = sim_scores.mean(axis=1)

In [21]:
average_scores.columns = ['index', 'score']

In [25]:
average_scores = average_scores.sort_values(ascending=False)

In [26]:
article_indices = average_scores.nlargest(10).keys()

In [27]:
article_indices

Int64Index([8, 7, 1, 5, 9, 4, 20, 21, 35, 33], dtype='int64')

In [30]:
data['url'].iloc[article_indices]

8     https://archiv.ihned.cz/c1-66322840-firmy-se-n...
7     https://byznys.ihned.cz/c1-66334240-stret-j-t-...
1     https://archiv.ihned.cz/c1-66333760-komunistic...
5     https://zahranicni.ihned.cz/c1-66334250-plukov...
9     https://byznys.ihned.cz/c1-66333170-postavit-n...
4     https://ego.ihned.cz/c1-66333210-architekturou...
20    https://archiv.ihned.cz/c1-66332880-kdyz-mam-n...
21    https://archiv.ihned.cz/c1-66333560-sloupek-mi...
35    https://byznys.ihned.cz/c1-66329440-podle-mini...
33    https://byznys.ihned.cz/c1-66327650-prilis-reg...
Name: url, dtype: object